In [1]:
import geopandas as gpd
import pandas as pd
from osgeo import ogr,gdal
import os
import xarray as xr
import rasterio
import numpy as np
import pyproj
from pygeos import from_wkb,from_wkt
import pygeos
from tqdm import tqdm
from shapely.wkb import loads
from pathlib import Path
import glob
from shapely.geometry import mapping
pd.options.mode.chained_assignment = None
from rasterio.mask import mask
import rioxarray
import matplotlib.pyplot as plt
from scipy import integrate

import warnings
warnings.filterwarnings("ignore")

C:\Users\mye500\AppData\Local\Temp\ipykernel_2448\167061698.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
gdal.SetConfigOption("OSM_CONFIG_FILE", os.path.join('..',"osmconf.ini"))

# change paths to make it work on your own machine
data_path = os.path.join('C:\\','Data','pg_risk_analysis')
tc_path = os.path.join(data_path,'tc_netcdf')
fl_path = os.path.join(data_path,'GLOFRIS')
osm_data_path = os.path.join('C:\\','Data','country_osm')
pg_data_path = os.path.join(data_path,'pg_data')
vul_curve_path = os.path.join(data_path,'vulnerability_curves','input_vulnerability_data.xlsx')
output_path = os.path.join('C:\\','projects','pg_risk_analysis','output')

In [3]:
def query_b(geoType,keyCol,**valConstraint):
    """
    This function builds an SQL query from the values passed to the retrieve() function.
    Arguments:
         *geoType* : Type of geometry (osm layer) to search for.
         *keyCol* : A list of keys/columns that should be selected from the layer.
         ***valConstraint* : A dictionary of constraints for the values. e.g. WHERE 'value'>20 or 'value'='constraint'
    Returns:
        *string: : a SQL query string.
    """
    query = "SELECT " + "osm_id"
    for a in keyCol: query+= ","+ a  
    query += " FROM " + geoType + " WHERE "
    # If there are values in the dictionary, add constraint clauses
    if valConstraint: 
        for a in [*valConstraint]:
            # For each value of the key, add the constraint
            for b in valConstraint[a]: query += a + b
        query+= " AND "
    # Always ensures the first key/col provided is not Null.
    query+= ""+str(keyCol[0]) +" IS NOT NULL" 
    return query 


def retrieve(osm_path,geoType,keyCol,**valConstraint):
    """
    Function to extract specified geometry and keys/values from OpenStreetMap
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.     
        *geoType* : Type of Geometry to retrieve. e.g. lines, multipolygons, etc.
        *keyCol* : These keys will be returned as columns in the dataframe.
        ***valConstraint: A dictionary specifiying the value constraints.  
        A key can have multiple values (as a list) for more than one constraint for key/value.  
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all columns, geometries, and constraints specified.    
    """
    driver=ogr.GetDriverByName('OSM')
    data = driver.Open(osm_path)
    query = query_b(geoType,keyCol,**valConstraint)
    sql_lyr = data.ExecuteSQL(query)
    features =[]
    # cl = columns 
    cl = ['osm_id'] 
    for a in keyCol: cl.append(a)
    if data is not None:
        print('query is finished, lets start the loop')
        for feature in tqdm(sql_lyr,desc='extract'):
            #try:
            if feature.GetField(keyCol[0]) is not None:
                geom1 = (feature.geometry().ExportToWkt())
                #print(geom1)
                geom = from_wkt(feature.geometry().ExportToWkt()) 
                if geom is None:
                    continue
                # field will become a row in the dataframe.
                field = []
                for i in cl: field.append(feature.GetField(i))
                field.append(geom)   
                features.append(field)
            #except:
            #    print("WARNING: skipped OSM feature")   
    else:
        print("ERROR: Nonetype error when requesting SQL. Check required.")    
    cl.append('geometry')                   
    if len(features) > 0:
        return pd.DataFrame(features,columns=cl)
    else:
        print("WARNING: No features or No Memory. returning empty GeoDataFrame") 
        return pd.DataFrame(columns=['osm_id','geometry'])

def power_polyline(osm_path):
    """
    Function to extract all energy linestrings from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique energy linestrings.
    """
    df = retrieve(osm_path,'lines',['power','voltage'])
    
    df = df.reset_index(drop=True).rename(columns={'power': 'asset'})
    
    #print(df) #check infra keys
    
    return df.reset_index(drop=True)

def power_polygon(osm_path): # check with joel, something was wrong here with extracting substations
    """
    Function to extract energy polygons from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique energy linestrings.
    """
    df = retrieve(osm_path,'multipolygons',['other_tags']) 
    
    df = df.loc[(df.other_tags.str.contains('power'))]   #keep rows containing power data         
    df = df.reset_index(drop=True).rename(columns={'other_tags': 'asset'})     
    
    df['asset'].loc[df['asset'].str.contains('"power"=>"substation"', case=False)]  = 'substation' #specify row
    df['asset'].loc[df['asset'].str.contains('"power"=>"plant"', case=False)] = 'plant' #specify row
    
    df = df.loc[(df.asset == 'substation') | (df.asset == 'plant')]
            
    return df.reset_index(drop=True) 

def electricity(osm_path):
    """
    Function to extract building polygons from OpenStreetMap    
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with all unique building polygons.    
    """
    df = retrieve(osm_path,'multipolygons',['power'])
    
    df = df.reset_index(drop=True).rename(columns={'power': 'asset'})
    
    #df = df[df.asset!='generator']
    df['asset'].loc[df['asset'].str.contains('"power"=>"substation"', case=False)]  = 'substation' #specify row
    df['asset'].loc[df['asset'].str.contains('"power"=>"plant"', case=False)] = 'plant' #specify row
    
    #print(df)  #check infra keys
    
    df = df.loc[(df.asset == 'substation') | (df.asset == 'plant')]
    
    return df.reset_index(drop=True)

def retrieve_poly_subs(osm_path, w_list, b_list):
    """
    Function to extract electricity substation polygons from OpenStreetMap
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region
        for which we want to do the analysis.
        *w_list* :  white list of keywords to search in the other_tags columns
        *b_list* :  black list of keywords of rows that should not be selected
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique substation.
    """
    df = retrieve(osm_path,'multipolygons',['other_tags'])
    df = df[df.other_tags.str.contains('substation', case=False, na=False)]
    #df = df.loc[(df.other_tags.str.contains('substation'))]
    df = df[~df.other_tags.str.contains('|'.join(b_list))]
    #df = df.reset_index(drop=True).rename(columns={'other_tags': 'asset'})
    df['asset']  = 'substation' #specify row
    #df = df.loc[(df.asset == 'substation')] #specify row
    return df.reset_index(drop=True)

def power_point(osm_path):
    """
    Function to extract energy points from OpenStreetMap  
    Arguments:
        *osm_path* : file path to the .osm.pbf file of the region 
        for which we want to do the analysis.        
    Returns:
        *GeoDataFrame* : a geopandas GeoDataFrame with specified unique energy linestrings.
    """   
    df = retrieve(osm_path,'points',['other_tags']) 
    df = df.loc[(df.other_tags.str.contains('power'))]  #keep rows containing power data       
    df = df.reset_index(drop=True).rename(columns={'other_tags': 'asset'})     
        
    df['asset'].loc[df['asset'].str.contains('"power"=>"tower"', case=False)]  = 'power_tower' #specify row
    df['asset'].loc[df['asset'].str.contains('"power"=>"pole"', case=False)] = 'power_pole' #specify row
    #df['asset'].loc[df['asset'].str.contains('"utility"=>"power"', case=False)] = 'power_tower' #specify row
    
    df = df.loc[(df.asset == 'power_tower') | (df.asset == 'power_pole')]
            
    return df.reset_index(drop=True)

In [4]:
def reproject(df_ds, current_crs="epsg:4326", approximate_crs="epsg:3857"):
    # Extract the input geometries as a numpy array of coordinates
    geometries = df_ds['geometry']
    coords = pygeos.get_coordinates(geometries)

    # Transform the coordinates using pyproj
    transformer = pyproj.Transformer.from_crs(current_crs, approximate_crs, always_xy=True)
    new_coords = transformer.transform(coords[:, 0], coords[:, 1])

    # Create a new GeoSeries with the reprojected coordinates
    return pygeos.set_coordinates(geometries.copy(), np.array(new_coords).T)

def buffer_assets(assets, buffer_size=100):
    """
    Create a buffer of a specified size around the geometries in a GeoDataFrame.
    
    Args:
        assets (GeoDataFrame): A GeoDataFrame containing geometries to be buffered.
        buffer_size (int, optional): The distance in the units of the GeoDataFrame's CRS to buffer the geometries.
            Defaults to 100.
    
    Returns:
        GeoDataFrame: A new GeoDataFrame with an additional column named 'buffered' containing the buffered
            geometries.
    """
    # Create a buffer of the specified size around the geometries
    assets['buffered'] = pygeos.buffer(assets.geometry.values, buffer_size)
    
    return assets

def load_curves_maxdam(vul_curve_path,hazard_type):
    """[summary]

    Args:
        data_path ([type]): [description]

    Returns:
        [type]: [description]
    """

    if hazard_type == 'tc':
        sheet_name = 'wind_curves'
    
    elif hazard_type == 'fl':
        sheet_name = 'flooding_curves'
    
    # load curves and maximum damages as separate inputs
    curves = pd.read_excel(vul_curve_path,sheet_name=sheet_name,skiprows=11,index_col=[0])
    
    if hazard_type == 'fl':
        maxdam = pd.read_excel(vul_curve_path,sheet_name=sheet_name,index_col=[0]).iloc[:8]
    elif hazard_type == 'tc':
        maxdam = pd.read_excel(vul_curve_path,sheet_name=sheet_name,index_col=[0],header=[0,1]).iloc[:8]
        maxdam = maxdam.rename({'substation_point':'substation'},level=0,axis=1)
            
    curves.columns = maxdam.columns
        
    #transpose maxdam so its easier work with the dataframe
    maxdam = maxdam.T

    #interpolate the curves to fill missing values
    curves = curves.interpolate()
    
    #print(curves)
   
    return curves,maxdam


def overlay_hazard_assets(df_ds, assets):
    """
    Overlay a set of assets with a hazard dataset and return the subset of assets that intersect with
    one or more hazard polygons or lines.
    
    Args:
        df_ds (GeoDataFrame): A GeoDataFrame containing the hazard dataset.
        assets (GeoDataFrame): A GeoDataFrame containing the assets to be overlaid with the hazard dataset.
    
    Returns:
        ndarray: A numpy array of integers representing the indices of the hazard geometries that intersect with
            the assets. If the assets have a 'buffered' column, the buffered geometries are used for the overlay.
    """
    hazard_tree = pygeos.STRtree(df_ds.geometry.values)
    if (pygeos.get_type_id(assets.iloc[0].geometry) == 3) | (pygeos.get_type_id(assets.iloc[0].geometry) == 6):
        return  hazard_tree.query_bulk(assets.geometry,predicate='intersects')    
    else:
        return  hazard_tree.query_bulk(assets.buffered,predicate='intersects')
    
def get_damage_per_asset_per_rp(asset,df_ds,assets,curves,maxdam,return_period,country):
    """
    Calculates the damage per asset per return period based on asset type, hazard curves and maximum damage

    Args:
        asset (tuple): Tuple with two dictionaries, containing the asset index and the hazard point index of the asset
        df_ds (pandas.DataFrame): A pandas DataFrame containing hazard points with a 'geometry' column
        assets (geopandas.GeoDataFrame): A GeoDataFrame containing asset geometries and asset type information
        curves (dict): A dictionary with the asset types as keys and their corresponding hazard curves as values
        maxdam (pandas.DataFrame): A pandas DataFrame containing the maximum damage for each asset type
        return_period (str): The return period for which the damage should be calculated
        country (str): The country for which the damage should be calculated

    Returns:
        list or tuple: Depending on the input, the function either returns a list of tuples with the asset index, the curve name and the calculated damage, or a tuple with None, None, None if no hazard points are found
    """
    
    # find the exact hazard overlays:
    get_hazard_points = df_ds.iloc[asset[1]['hazard_point'].values].reset_index()
    get_hazard_points = get_hazard_points.loc[pygeos.intersects(get_hazard_points.geometry.values,assets.iloc[asset[0]].geometry)]

    
    asset_type = assets.iloc[asset[0]].asset
    asset_geom = assets.iloc[asset[0]].geometry

    if asset_type in ['plant','substation','generator']:
        maxdam_asset = maxdam.loc[asset_type].MaxDam/pygeos.area(asset_geom)
        lowerdam_asset = maxdam.loc[asset_type].LowerDam/pygeos.area(asset_geom)
        upperdam_asset = maxdam.loc[asset_type].UpperDam/pygeos.area(asset_geom)
    else:
        maxdam_asset = maxdam.loc[asset_type].MaxDam
        lowerdam_asset = maxdam.loc[asset_type].LowerDam
        upperdam_asset = maxdam.loc[asset_type].UpperDam


    hazard_intensity = curves[asset_type].index.values
    
    if isinstance(curves[asset_type],pd.core.series.Series):
        fragility_values = curves[asset_type].values.flatten()
        only_one = True
        curve_name = curves[asset_type].name
    elif len(curves[asset_type].columns) == 1:
        fragility_values = curves[asset_type].values.flatten()      
        only_one = True   
        curve_name = curves[asset_type].columns[0]
    else:
        fragility_values = curves[asset_type].values#.T[0]
        maxdam_asset = maxdam_asset.values#[0]
        only_one = False

    if len(get_hazard_points) == 0:
        return [return_period,asset[0],None,None]
    else:
        if only_one:    
            # run the calculation as normal when the asset just has a single curve
            if pygeos.get_type_id(asset_geom) == 1:            
                get_hazard_points['overlay_meters'] = pygeos.length(pygeos.intersection(get_hazard_points.geometry.values,asset_geom))
                return [return_period,asset[0],curve_name,np.sum((np.interp(get_hazard_points[return_period].values,hazard_intensity,
                                                             fragility_values))*get_hazard_points.overlay_meters*maxdam_asset),
                                                          np.sum((np.interp(get_hazard_points[return_period].values,hazard_intensity,
                                                             fragility_values))*get_hazard_points.overlay_meters*lowerdam_asset),
                                                          np.sum((np.interp(get_hazard_points[return_period].values,hazard_intensity,
                                                             fragility_values))*get_hazard_points.overlay_meters*upperdam_asset)]

            elif (pygeos.get_type_id(asset_geom) == 3) | (pygeos.get_type_id(asset_geom) == 6) :
                get_hazard_points['overlay_m2'] = pygeos.area(pygeos.intersection(get_hazard_points.geometry.values,asset_geom))
                return [return_period,asset[0],curve_name,get_hazard_points.apply(lambda x: np.interp(x[return_period],hazard_intensity, 
                                                                  fragility_values)*maxdam_asset*x.overlay_m2,axis=1).sum(),
                                                          get_hazard_points.apply(lambda x: np.interp(x[return_period],hazard_intensity, 
                                                                  fragility_values)*lowerdam_asset*x.overlay_m2,axis=1).sum(),
                                                          get_hazard_points.apply(lambda x: np.interp(x[return_period],hazard_intensity, 
                                                                  fragility_values)*upperdam_asset*x.overlay_m2,axis=1).sum()]  

            else:
                return [return_period,asset[0],curve_name,np.sum((np.interp(get_hazard_points[return_period].values,
                                                             hazard_intensity,fragility_values))*maxdam_asset),
                                                          np.sum((np.interp(get_hazard_points[return_period].values,
                                                             hazard_intensity,fragility_values))*lowerdam_asset),
                                                          np.sum((np.interp(get_hazard_points[return_period].values,
                                                             hazard_intensity,fragility_values))*upperdam_asset)]
        else:
            # run the calculation when the asset has multiple curves
            if pygeos.get_type_id(asset_geom) == 1:            
                get_hazard_points['overlay_meters'] = pygeos.length(pygeos.intersection(get_hazard_points.geometry.values,asset_geom))
            elif (pygeos.get_type_id(asset_geom) == 3) | (pygeos.get_type_id(asset_geom) == 6) :
                get_hazard_points['overlay_m2'] = pygeos.area(pygeos.intersection(get_hazard_points.geometry.values,asset_geom))
            
            collect_all = []
            for iter_,curve_ids in enumerate(curves[asset_type].columns):
                if pygeos.get_type_id(asset_geom) == 1:
                    collect_all.append([return_period,asset[0],curves[asset_type].columns[iter_],
                                        np.sum((np.interp(get_hazard_points[return_period].values,
                                                          hazard_intensity,fragility_values.T[iter_]))*get_hazard_points.overlay_meters*maxdam_asset[iter_]),
                                        np.sum((np.interp(get_hazard_points[return_period].values,
                                                          hazard_intensity,fragility_values.T[iter_]))*get_hazard_points.overlay_meters*lowerdam_asset[iter_]),
                                        np.sum((np.interp(get_hazard_points[return_period].values,
                                                          hazard_intensity,fragility_values.T[iter_]))*get_hazard_points.overlay_meters*upperdam_asset[iter_])])
                                   
                elif (pygeos.get_type_id(asset_geom) == 3) | (pygeos.get_type_id(asset_geom) == 6) :
                    collect_all.append([return_period,asset[0],curves[asset_type].columns[iter_],
                                        get_hazard_points.apply(lambda x: np.interp(x[return_period], hazard_intensity,
                                                                                    fragility_values.T[iter_])*maxdam_asset[iter_]*x.overlay_m2,axis=1).sum(),
                                        get_hazard_points.apply(lambda x: np.interp(x[return_period], hazard_intensity,
                                                                                    fragility_values.T[iter_])*lowerdam_asset[iter_]*x.overlay_m2,axis=1).sum(),
                                        get_hazard_points.apply(lambda x: np.interp(x[return_period], hazard_intensity,
                                                                                    fragility_values.T[iter_])*upperdam_asset[iter_]*x.overlay_m2,axis=1).sum()])

                else:
                    collect_all.append([return_period,asset[0],curves[asset_type].columns[iter_],
                                        np.sum((np.interp(get_hazard_points[return_period].values,
                                                          hazard_intensity,fragility_values.T[iter_]))*maxdam_asset[iter_]),
                                        np.sum((np.interp(get_hazard_points[return_period].values,
                                                          hazard_intensity,fragility_values.T[iter_]))*lowerdam_asset[iter_]),
                                        np.sum((np.interp(get_hazard_points[return_period].values,
                                                          hazard_intensity,fragility_values.T[iter_]))*upperdam_asset[iter_])])
            return collect_all

In [5]:
load_curves_maxdam(vul_curve_path,'fl')[1]

Infrastructure type,Code,Type vulnerability data,Unit,Specific occupancy,Reference,MaxDam,LowerDam,UpperDam
plant,F1_1_1,curve,euro/facility,Small power plants,"FEMA, 2021",154165240.329434,115623930.247075,192706550.411792
plant.1,F1_1_2,curve,euro/facility,Medium power plants,"FEMA, 2021",775256237.288818,581442177.966614,969070296.611023
plant.2,F1_1_3,curve,euro/facility,Large power plants,"FEMA, 2021",775256237.288818,581442177.966614,969070296.611023
substation,F2_1_1,curve,euro/facility,low votage,"FEMA, 2021",8860071.283301,6645053.462476,11075089.104126
substation.1,F2_1_2,curve,euro/facility,medium votage,"FEMA, 2021",17720142.566602,13290106.924951,22150178.208252
substation.2,F2_1_3,curve,euro/facility,high votage,"FEMA, 2021",44300356.416504,33225267.312378,55375445.52063
power_tower,F3_1,curve,euro/facility,NaN,"FEMA, 2013",117360.574259,88020.430694,146700.717823
power_pole,F4_1_1,curve,euro/facility,wood,"FEMA, 2013",56195.614965,42146.711224,70244.518706
power_pole.1,F4_1_2,curve,euro/facility,concrete,"FEMA, 2013",57458.437773,43093.82833,71823.047217
power_pole.2,F4_1_3,curve,euro/facility,steel monopole,"FEMA, 2013",90291.830787,67718.87309,112864.788483


In [5]:
def open_storm_data(country_code):
    # list of available climate models
    climate_models = ['','_CMCC-CM2-VHR4','_CNRM-CM6-1-HR','_EC-Earth3P-HR','_HadGEM3-GC31-HM']

    # dictionary of basins for each country
    country_basin = {
        "BRN": ["WP"],
        "KHM": ["WP"],
        "CHN": ["WP", "NI"],
        "IDN": ["SI", "SP", "NI", "WP"],
        "JPN": ["WP"],
        "LAO": ["WP"],
        "MYS": ["WP", "NI"],
        "MNG": ["WP", "NI"],
        "MMR": ["NI", "WP"],
        "PRK": ["WP"],
        "PHL": ["WP"],
        "SGP": ["WP"],
        "KOR": ["WP"],
        "TWN": ["WP"],
        "THA": ["WP", "NI"],
        "VNM": ["WP"]
    }

    # load country geometry file and create geometry to clip
    ne_countries = gpd.read_file(os.path.join(data_path,'..',"natural_earth","ne_10m_admin_0_countries.shp"))
    bbox = ne_countries.loc[ne_countries['ISO_A3']==country_code].geometry.buffer(1).values[0].bounds

    df_ds = {}
    for climate_model in climate_models:
        concat_prep = []

        #combine STORM data from different basins
        if "WP" in country_basin[country_code]:
            WP = load_storm_data(climate_model,'WP',bbox)
            concat_prep.append(WP)
        if "SP" in country_basin[country_code]:
            SP = load_storm_data(climate_model,'SP',bbox)
            concat_prep.append(SP)
        if "NI" in country_basin[country_code]:            
            NI = load_storm_data(climate_model,'NI',bbox)
            concat_prep.append(NI)            
        if "SI" in country_basin[country_code]:       
            SI = load_storm_data(climate_model,'SI',bbox)
            concat_prep.append(SI)            
                   
        df_ds_cl = pd.concat(concat_prep, keys=country_basin[country_code])
        df_ds_cl = df_ds_cl.reset_index(drop=True)
        df_ds[climate_model] = df_ds_cl

    return df_ds

def load_storm_data(climate_model,basin,bbox):

    filename = os.path.join(tc_path, f'STORM_FIXED_RETURN_PERIODS{climate_model}_{basin}.nc')
    
    # load data from NetCDF file
    with xr.open_dataset(filename) as ds:
        
        # convert data to WGS84 CRS
        ds.rio.write_crs(4326, inplace=True)
        ds = ds.rio.clip_box(minx=bbox[0], miny=bbox[1], maxx=bbox[2], maxy=bbox[3])
        
        # get the mean values
        df_ds = ds['mean'].to_dataframe().unstack(level=2).reset_index()

        # create geometry values and drop lat lon columns
        df_ds['geometry'] = [pygeos.points(x) for x in list(zip(df_ds['lon'], df_ds['lat']))]
        df_ds = df_ds.drop(['lat', 'lon'], axis=1, level=0)
        
        # interpolate wind speeds of 1, 2, and 5-yr return period
        ## rename columns to return periods (must be integer for interpolating)
        df_ds_geometry = pd.DataFrame()
        df_ds_geometry['geometry'] = df_ds['geometry']
        df_ds = df_ds.drop(['geometry'], axis=1, level=0)
        df_ds = df_ds['mean']
        df_ds.columns = [int(x) for x in ds['mean']['rp']]
        df_ds[1] = np.nan
        df_ds[2] = np.nan
        df_ds[5] = np.nan
        df_ds[25] = np.nan
        df_ds[250] = np.nan
        df_ds = df_ds.reindex(sorted(df_ds.columns), axis=1)
        df_ds = df_ds.interpolate(method='linear', axis=1, limit_direction='both')
        df_ds['geometry'] = df_ds_geometry['geometry']
        df_ds = df_ds[[1, 2, 5, 10, 25, 50, 100, 250, 500, 1000, 'geometry']]
        
        # rename columns to return periods
        df_ds.columns = ['1_{}{}'.format(int(x), climate_model) for x in [1, 2, 5, 10, 25, 50, 100, 250, 500, 1000]] +['geometry']     
        df_ds['geometry'] = pygeos.buffer(df_ds.geometry, radius=0.1/2, cap_style='square').values
        
        # reproject the geometry column to the specified CRS
        df_ds['geometry'] = reproject(df_ds)
            
        # drop all non values to reduce size
        #df_ds = df_ds.loc[~df_ds['1_10000{}'.format(climate_model)].isna()].reset_index(drop=True)
        df_ds = df_ds.fillna(0)

    return df_ds

In [7]:
twn_wind=open_storm_data('TWN')
print(type(twn_wind))

<class 'dict'>


In [8]:
%%time
open_storm_data('TWN')['']

CPU times: total: 2.89 s
Wall time: 2.88 s


,1_1,1_2,1_5,1_10,1_25,1_50,1_100,1_250,1_500,1_1000,geometry
0,31.484970,31.484970,31.484970,31.484970,34.882271,37.150791,38.786932,41.029938,42.933264,43.897242,"POLYGON ((13057776.27 2391878.588, 13057776.27..."
1,31.561684,31.561684,31.561684,31.561684,34.941517,37.417290,38.952814,40.987155,42.433489,43.846372,"POLYGON ((13068908.219 2391878.588, 13068908.2..."
2,31.559660,31.559660,31.559660,31.559660,34.989208,37.441990,39.295692,41.087890,42.636647,43.937314,"POLYGON ((13080040.168 2391878.588, 13080040.1..."
3,31.649051,31.649051,31.649051,31.649051,35.093386,37.464786,39.427878,41.296611,42.939086,44.289626,"POLYGON ((13091172.117 2391878.588, 13091172.1..."
4,31.749391,31.749391,31.749391,31.749391,35.266336,37.634156,39.579647,41.514516,42.741313,44.298240,"POLYGON ((13102304.066 2391878.588, 13102304.0..."
...,...,...,...,...,...,...,...,...,...,...,...
3181,35.069860,35.069860,35.069860,35.069860,39.243768,42.045388,44.128518,46.371252,48.097373,49.527067,"POLYGON ((13658901.52 3036284.923, 13658901.52..."
3182,35.159120,35.159120,35.159120,35.159120,39.261418,41.821312,44.056902,46.617187,48.180678,49.415934,"POLYGON ((13670033.469 3036284.923, 13670033.4..."
3183,35.238116,35.238116,35.238116,35.238116,39.250911,41.894078,44.101615,46.562302,47.930412,48.967779,"POLYGON ((13681165.418 3036284.923, 13681165.4..."
3184,35.309186,35.309186,35.309186,35.309186,39.248515,41.935010,44.177309,46.750433,47.926717,48.968906,"POLYGON ((13692297.368 3036284.923, 13692297.3..."


In [8]:
def clip_flood_data(country_code):
    
    # load country geometry file and create geometry to clip
    ne_countries = gpd.read_file(os.path.join(data_path,'..',"natural_earth","ne_10m_admin_0_countries.shp"))
    geometry = ne_countries.loc[ne_countries['ISO_A3']==country_code].geometry.values[0]
    geoms = [mapping(geometry)]
    
    #climate_model: historical, rcp4p5, rcp8p5; time_period: hist, 2030, 2050, 2080
    rps = ['0001','0002','0005','0010','0025','0050','0100','0250','0500','1000']
    climate_models = ['historical','rcp8p5']
    
    #"/scistor/ivm/data_catalogue/open_street_map/pg_risk_analysis/GLOFRIS/global/inuncoast_historical_nosub_hist_rp0001_0.tif"

    for rp in rps:
        #global input_file
        for climate_model in climate_models:
            if climate_model=='historical':
                #f rps=='0001':
                    input_file = os.path.join(fl_path,'global',
                                              'inuncoast_{}_nosub_hist_rp{}_0.tif'.format(climate_model,rp)) 
                #elif rps==['0002','0005','0010','0025','0050','0100','0250','0500','1000']:
                #    input_file = os.path.join(fl_path,'global',
                #                              'inuncoast_{}_nosub_hist_rp{}_0.tif'.format(climate_model,rp)) 
            elif climate_model=='rcp8p5':
                #f rps=='0001':
                    input_file = os.path.join(fl_path,'global',
                                              'inuncoast_{}_nosub_2030_rp{}_0.tif'.format(climate_model,rp))
                #elif rps==['0002','0005','0010','0025','0050','0100','0250','0500','1000']:
                #    input_file = os.path.join(fl_path,'global',
                #                              'inuncoast_{}_nosub_2030_rp{}_0.tif'.format(climate_model,rp))
            
            # load raster file and save clipped version
            with rasterio.open(input_file) as src:
                out_image, out_transform = mask(src, geoms, crop=True)
                out_meta = src.meta

                out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

                if 'scistor' in fl_path:
                    file_path = os.path.join(fl_path,'country','_'.join([country_code]+input_file.split('_')[6:]))
                else:
                    file_path = os.path.join(fl_path,'country','_'.join([country_code]+input_file.split('_')[3:]))

                with rasterio.open(file_path, "w", **out_meta) as dest:
                    dest.write(out_image)

def load_flood_data(country_code,climate_model):
     
    rps = ['0001','0002','0005','0010','0025','0050','0100','0250','0500','1000']
    collect_df_ds = []
    
    if climate_model=='historical':
        print('Loading historical coastal flood data ...')
        for rp in rps:
            #for file in files:
            file_path = os.path.join(fl_path,'country','{}_{}_nosub_hist_rp{}_0.tif'.format(country_code,climate_model,rp))
            with xr.open_dataset(file_path) as ds: #, engine="rasterio"
                df_ds = ds.to_dataframe().reset_index()
                df_ds['geometry'] = pygeos.points(df_ds.x,y=df_ds.y)
                df_ds = df_ds.rename(columns={'band_data': 'rp'+rp}) #rename to return period
                
                # move from meters to centimeters
                df_ds['rp'+rp] = (df_ds['rp'+rp]*100)         
                df_ds = df_ds.drop(['band','x', 'y','spatial_ref'], axis=1)
                df_ds = df_ds.dropna()
                df_ds = df_ds.reset_index(drop=True)
                df_ds.geometry= pygeos.buffer(df_ds.geometry,radius=0.00833/2,cap_style='square').values  #?????????????????????????
                df_ds['geometry'] = reproject(df_ds)
                collect_df_ds.append(df_ds)

        df_all = collect_df_ds[0].merge(collect_df_ds[1]).merge(collect_df_ds[2]).merge(collect_df_ds[3]).merge(collect_df_ds[4])\
                 .merge(collect_df_ds[5]).merge(collect_df_ds[6]).merge(collect_df_ds[7]).merge(collect_df_ds[8]).merge(collect_df_ds[9])
        df_all = df_all.loc[df_all['rp1000']>0].reset_index(drop=True)

    elif climate_model=='rcp8p5':
        print('Loading future coastal flood data ...')
        for rp in rps:
            #for file in files:
            file_path = os.path.join(fl_path,'country','{}_{}_nosub_2030_rp{}_0.tif'.format(country_code,climate_model,rp))
            with xr.open_dataset(file_path) as ds: #, engine="rasterio"
                df_ds = ds.to_dataframe().reset_index()
                df_ds['geometry'] = pygeos.points(df_ds.x,y=df_ds.y)
                df_ds = df_ds.rename(columns={'band_data': 'rp'+rp}) #rename to return period
                df_ds['rp'+rp] = (df_ds['rp'+rp]*100)
                df_ds = df_ds.drop(['band','x', 'y','spatial_ref'], axis=1)
                df_ds = df_ds.dropna()
                df_ds = df_ds.reset_index(drop=True)
                df_ds.geometry= pygeos.buffer(df_ds.geometry,radius=0.00833/2,cap_style='square').values
                df_ds['geometry'] = reproject(df_ds)
                collect_df_ds.append(df_ds)

        df_all = collect_df_ds[0].merge(collect_df_ds[1]).merge(collect_df_ds[2]).merge(collect_df_ds[3]).merge(collect_df_ds[4])\
                 .merge(collect_df_ds[5]).merge(collect_df_ds[6]).merge(collect_df_ds[7]).merge(collect_df_ds[8]).merge(collect_df_ds[9])

        df_all = df_all.loc[df_all['rp1000']>0].reset_index(drop=True)
    return df_all

def open_flood_data(country_code):
    climate_models = ['historical','rcp8p5']
    df_ds = {}
    for climate_model in climate_models:
        df_ds_sc = load_flood_data(country_code,climate_model)

        df_ds[climate_model] = df_ds_sc
    
    return df_ds

In [10]:
clip_flood_data('BRN')

In [ ]:
%%time
twn_flood = open_flood_data('CHN')

Loading historical coastal flood data ...


In [12]:
print(type(twn_flood))

<class 'dict'>


In [487]:
twn_flood#['historical']

{'historical':           rp0001                                           geometry  \
 0     176.420731  POLYGON ((13168167.913 2811396.474, 13168167.9...   
 1     190.201111  POLYGON ((13169095.575 2810377.258, 13169095.5...   
 2       0.000000  POLYGON ((13170023.238 2811396.474, 13170023.2...   
 3       0.000000  POLYGON ((13170023.238 2810377.258, 13170023.2...   
 4       0.000000  POLYGON ((13170950.9 2811396.474, 13170950.9 2...   
 ...          ...                                                ...   
 998   127.308418  POLYGON ((13525317.946 2521533.141, 13525317.9...   
 999     9.339666  POLYGON ((13526245.608 2519531.048, 13526245.6...   
 1000    0.000000  POLYGON ((13559641.456 2840983.226, 13559641.4...   
 1001    0.000000  POLYGON ((13560569.118 2840983.226, 13560569.1...   
 1002    0.000000  POLYGON ((13560569.118 2839962.04, 13560569.11...   
 
           rp0002      rp0005      rp0010      rp0025      rp0050      rp0100  \
 0     190.869278  226.426697  249.9687

# OSM data processing

In [13]:
def extract_osm_infrastructure(country_code,osm_data_path):

    # lines
    osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))
    power_lines_country = power_polyline(osm_path)
    power_lines_country['geometry'] = reproject(power_lines_country)
    power_lines_country = buffer_assets(power_lines_country.loc[power_lines_country.asset.isin(
        ['cable','minor_cable','line','minor_line'])],buffer_size=100).reset_index(drop=True)
    
    # polygons
    osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))
    power_poly_country = electricity(osm_path)
    power_poly_country['geometry'] = reproject(power_poly_country)
    
    # points
    osm_path = os.path.join(osm_data_path,'{}.osm.pbf'.format(country_code))
    power_points_country = power_point(osm_path)
    power_points_country['geometry'] = reproject(power_points_country)
    power_points_country = buffer_assets(power_points_country.loc[power_points_country.asset.isin(
        ['power_tower','power_pole'])],buffer_size=100).reset_index(drop=True)


    return power_lines_country,power_poly_country,power_points_country

In [15]:
osm_power_infra = extract_osm_infrastructure('BRN',osm_data_path)

query is finished, lets start the loop


extract: 100%|████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 442.36it/s]


query is finished, lets start the loop


extract: 100%|█████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 81.68it/s]


query is finished, lets start the loop


extract: 100%|██████████████████████████████████████████████████████████████████| 8774/8774 [00:00<00:00, 10040.31it/s]


In [78]:
print(type(osm_power_infra))

<class 'tuple'>


In [36]:
def assess_damage_osm(country_code,osm_power_infra,hazard_type):
    
    # load curves and maxdam
    curves,maxdam = load_curves_maxdam(vul_curve_path,hazard_type)
    
    # read infrastructure data:
    power_lines,power_poly,power_points = osm_power_infra

    if hazard_type=='tc':
        # read wind data
        climate_models = ['','_CMCC-CM2-VHR4','_CNRM-CM6-1-HR','_EC-Earth3P-HR','_HadGEM3-GC31-HM']#,'_CMCC-CM2-VHR4','_CNRM-CM6-1-HR','_EC-Earth3P-HR','_HadGEM3-GC31-HM']
        df_ds = open_storm_data(country_code)
        
        # remove assets that will not have any damage
        power_lines = power_lines.loc[power_lines.asset != 'cable'].reset_index(drop=True)
        power_poly = power_poly.loc[power_poly.asset != 'plant'].reset_index(drop=True)

    elif hazard_type=='fl':
        # read flood data
        climate_models = ['historical','rcp8p5']
        df_ds = open_flood_data(country_code) 
        
    #calculate damaged lines in loop by climate_model
    damaged_lines = {}
    damaged_poly = {}
    damaged_points = {}
    
    for climate_model in climate_models:
        
        if hazard_type == 'tc':
            return_periods = ['1_1{}'.format(climate_model),'1_2{}'.format(climate_model),'1_5{}'.format(climate_model),'1_10{}'.format(climate_model),
                              '1_25{}'.format(climate_model),'1_50{}'.format(climate_model),'1_100{}'.format(climate_model),
                              '1_250{}'.format(climate_model),'1_500{}'.format(climate_model),'1_1000{}'.format(climate_model)]
        elif hazard_type == 'fl':
            return_periods = ['rp0001','rp0002','rp0005','rp0010','rp0025','rp0050','rp0100','rp0250','rp0500','rp1000']  

        # assess damage for lines
        overlay_lines = pd.DataFrame(overlay_hazard_assets(df_ds[climate_model],power_lines).T,
                                     columns=['asset','hazard_point'])
        
        if len(overlay_lines) == 0:
            damaged_lines[climate_model] = pd.DataFrame()
            
        else:            
            collect_line_damages = []
            for asset in tqdm(overlay_lines.groupby('asset'),total=len(overlay_lines.asset.unique()),
                              desc='polyline damage calculation for {} {} ({})'.format(country_code,hazard_type,climate_model)):
                for return_period in return_periods:
                    collect_line_damages.append(get_damage_per_asset_per_rp(asset,
                                                                            df_ds[climate_model],
                                                                            power_lines,
                                                                            curves,
                                                                            maxdam,
                                                                            return_period,
                                                                            country_code))

            get_asset_type_line = dict(zip(power_lines.index,power_lines.asset))

            if hazard_type == 'fl':
                results = pd.DataFrame(collect_line_damages,columns=['rp','asset','curve','meandam','lowerdam','upperdam'])
            elif hazard_type == 'tc':
                results = pd.DataFrame([item for sublist in collect_line_damages 
                                        for item in sublist],columns=['rp','asset','curve','meandam','lowerdam','upperdam'])

            results['asset_type'] = results.asset.apply(lambda x : get_asset_type_line[x])

            damaged_lines[climate_model] = results.groupby(['rp','curve','asset_type']).sum().drop(['asset'], axis=1).reset_index()
            
        # assess damage for polygons
        if len(power_poly) > 0:
            overlay_poly = pd.DataFrame(overlay_hazard_assets(df_ds[climate_model],power_poly).T,
                                        columns=['asset','hazard_point'])
        else:
            overlay_poly = pd.DataFrame()
            
        if len(overlay_poly) == 0:
            damaged_poly[climate_model] = pd.DataFrame()
        
        else:
            collect_poly_damages = []
            for asset in tqdm(overlay_poly.groupby('asset'),total=len(overlay_poly.asset.unique()),
                              desc='polygon damage calculation for {} {} ({})'.format(country_code,hazard_type,climate_model)):
                for return_period in return_periods:
                    collect_poly_damages.append(get_damage_per_asset_per_rp(asset,
                                                                            df_ds[climate_model],
                                                                            power_poly,
                                                                            curves,
                                                                            maxdam,
                                                                            return_period,
                                                                            country_code))

            get_asset_type_poly = dict(zip(power_poly.index,power_poly.asset))

            if hazard_type == 'fl':
                results = pd.DataFrame(collect_poly_damages ,columns=['rp','asset','curve','meandam','lowerdam','upperdam'])
            elif hazard_type == 'tc':
                results = pd.DataFrame([item for sublist in collect_poly_damages 
                                        for item in sublist],columns=['rp','asset','curve','meandam','lowerdam','upperdam'])
                
            results['asset_type'] = results.asset.apply(lambda x : get_asset_type_poly[x])

            damaged_poly[climate_model] = results.groupby(['rp','curve','asset_type']).sum().drop(['asset'], axis=1).reset_index()

        # assess damage for points
        overlay_points = pd.DataFrame(overlay_hazard_assets(df_ds[climate_model],power_points).T,
                                      columns=['asset','hazard_point'])
        
        if len(overlay_points) == 0:
            damaged_points[climate_model] = pd.DataFrame()
            
        else:
            collect_point_damages = []
            for asset in tqdm(overlay_points.groupby('asset'),total=len(overlay_points.asset.unique()),
                              desc='point damage calculation for {} {} ({})'.format(country_code,hazard_type,climate_model)):
                for return_period in return_periods:
                    collect_point_damages.append(get_damage_per_asset_per_rp(asset,
                                                                             df_ds[climate_model],
                                                                             power_points,
                                                                             curves,
                                                                             maxdam,
                                                                             return_period,
                                                                             country_code))

            get_asset_type_point = dict(zip(power_points.index,power_points.asset))

            if hazard_type == 'fl':
                results = pd.DataFrame(collect_point_damages ,columns=['rp','asset','curve','meandam','lowerdam','upperdam'])
            elif hazard_type == 'tc':
                results = pd.DataFrame([item for sublist in collect_point_damages 
                                        for item in sublist],columns=['rp','asset','curve','meandam','lowerdam','upperdam'])

            results['asset_type'] = results.asset.apply(lambda x : get_asset_type_point[x])    

            damaged_points[climate_model] = results.groupby(['rp','curve','asset_type']).sum().drop(['asset'], axis=1).reset_index()

    return damaged_lines,damaged_poly,damaged_points

In [37]:
osm_damage_infra = assess_damage_osm('BRN',osm_power_infra,'fl')

Loading historical coastal flood data ...
Loading future coastal flood data ...


In [53]:
osm_damage_infra[2]['historical']

""


In [23]:
# extract nested dict by key
osm_damage_infra[0]#['historical']

{}

In [28]:
print(type(osm_damage_infra[0]))

<class 'dict'>


In [54]:
def country_analysis_osm(country_code,hazard_type): #

    # extract infrastructure data from OSM
    osm_power_infra = extract_osm_infrastructure(country_code,osm_data_path)
    
    # assess damage to hazard_type
    osm_damage_infra = assess_damage_osm(country_code,osm_power_infra,hazard_type)
    
    if hazard_type=='tc':
        climate_models = ['','_CMCC-CM2-VHR4','_CNRM-CM6-1-HR','_EC-Earth3P-HR','_HadGEM3-GC31-HM']
    
    elif hazard_type=='fl':
        climate_models = ['historical','rcp8p5']

    line_risk = {}
    plant_risk = {}
    substation_risk = {}
    tower_risk = {}
    pole_risk = {}
    
    for i in range(len(osm_damage_infra)):
        for climate_model in climate_models:
            df = osm_damage_infra[i][climate_model]
                
            if len(df) == 0:
                print("No {}_{} risk of infra_type {} in {}".format(hazard_type,climate_model,i,country_code))

            else:
                with pd.ExcelWriter(os.path.join(output_path,'damage','{}_osm_{}_{}_damage_{}'.format(country_code,hazard_type,climate_model,i)+'.xlsx')) as writer:
                    df.to_excel(writer)

                if hazard_type == 'tc':
                    df['rp'] = df['rp'].replace(['1_1{}'.format(climate_model),'1_2{}'.format(climate_model),'1_5{}'.format(climate_model),
                                                 '1_10{}'.format(climate_model),'1_25{}'.format(climate_model),'1_50{}'.format(climate_model),
                                                 '1_100{}'.format(climate_model),'1_250{}'.format(climate_model),'1_500{}'.format(climate_model),
                                                 '1_1000{}'.format(climate_model)],
                                                [1,0.5,0.2,0.1,0.04,0.02,0.01,0.004,0.002,0.001])

                elif hazard_type == 'fl':
                    df['rp'] = df['rp'].replace(['rp0001','rp0002','rp0005','rp0010','rp0025','rp0050','rp0100','rp0250','rp0500','rp1000'],
                                                [1,0.5,0.2,0.1,0.04,0.02,0.01,0.004,0.002,0.001])

                #assess risk for power lines
                if i == 0:
                    loss_list = df.meandam.values.tolist()
                    RPS = df.rp.values.tolist()
                    line_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                #assess risk for power plants and substations
                elif i == 1:
                    loss_list = df.loc[df['asset_type'] == 'plant']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'plant']
                    RPS = RPS.rp.values.tolist()
                    plant_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                    loss_list = df.loc[df['asset_type'] == 'substation']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'substation']
                    RPS = RPS.rp.values.tolist()
                    substation_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                #assess risk for power towers and power poles
                elif i == 2:
                    loss_list = df.loc[df['asset_type'] == 'power_tower']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'power_tower']
                    RPS = RPS.rp.values.tolist()
                    tower_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                    loss_list = df.loc[df['asset_type'] == 'power_pole']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'power_pole']
                    RPS = RPS.rp.values.tolist()
                    pole_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])
                
    return line_risk,plant_risk,substation_risk,tower_risk,pole_risk

In [55]:
%%time
osm_risk = country_analysis_osm('BRN','fl')

query is finished, lets start the loop


extract: 100%|████████████████████████████████████████████████████████████████████████| 87/87 [00:00<00:00, 483.20it/s]


query is finished, lets start the loop


extract: 100%|█████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 72.26it/s]


query is finished, lets start the loop


extract: 100%|███████████████████████████████████████████████████████████████████| 8774/8774 [00:00<00:00, 9890.56it/s]


Loading historical coastal flood data ...
Loading future coastal flood data ...
No fl_historical risk of infra_type 0 in BRN
No fl_rcp8p5 risk of infra_type 0 in BRN
No fl_historical risk of infra_type 1 in BRN
No fl_rcp8p5 risk of infra_type 1 in BRN
No fl_historical risk of infra_type 2 in BRN
No fl_rcp8p5 risk of infra_type 2 in BRN
CPU times: total: 13.1 s
Wall time: 13.1 s


In [634]:
osm_risk

({'historical': 105424.89081186829, 'rcp8p5': 136900.6743702326},
 {'historical': 1956667104.920839, 'rcp8p5': 2598842791.7569385},
 {'historical': 5313279.909809661, 'rcp8p5': 6924928.7416341975},
 {'historical': 145221.70252471452, 'rcp8p5': 197047.65002692994},
 {'historical': 0.0, 'rcp8p5': 0.19539434541201794})

# Government data processing

In [615]:
# load collected power grid data
def extract_pg_data(country_code,pg_type):
    
    files = [x for x in os.listdir(pg_data_path)  if country_code in x ]
    
    if pg_type=='line':
        for file in files: 
            file_path = os.path.join(pg_data_path,'{}_{}.gpkg'.format(country_code,pg_type))

            pg_data_country = gpd.read_file(file_path)
            pg_data_country = pd.DataFrame(pg_data_country.copy())
            #print(pg_data_country.head())
            pg_data_country.geometry = pygeos.from_shapely(pg_data_country.geometry)
            pg_data_country['geometry'] = reproject(pg_data_country)

        pg_data_country = buffer_assets(pg_data_country.loc[pg_data_country.asset.isin(['line'])],
                                        buffer_size=100).reset_index(drop=True)

    elif pg_type=='point':
        for file in files:
            file_path = os.path.join(pg_data_path,'{}_{}.gpkg'.format(country_code,pg_type))
                
            pg_data_country = gpd.read_file(file_path)
            pg_data_country = pd.DataFrame(pg_data_country.copy())
            pg_data_country.geometry = pygeos.from_shapely(pg_data_country.geometry)
            pg_data_country['geometry'] = reproject(pg_data_country)
            #print(pg_data_country)

        pg_data_country = buffer_assets(pg_data_country.loc[pg_data_country.asset.isin(['plant_point','substation_point','power_tower','power_pole'])],
                                        buffer_size=100).reset_index(drop=True)

    return pg_data_country

def open_pg_data(country_code):
    pg_lines = extract_pg_data(country_code,'line')
    pg_points = extract_pg_data(country_code,'point')

    return pg_lines,pg_points

In [616]:
pg_infra = open_pg_data('LAO')
print(type(pg_infra))
pg_infra[0]

<class 'tuple'>


,status,capacity_kV,value,id,source,country,operator,undergrnd,phases,cables,year,asset,geometry,buffered
0,Existing,230,transmission_line,0.0,World Bank,Laos,None,None,None,None,None,line,"LINESTRING (11642041.532 2064458.971, 11641491...","POLYGON ((11641591.898 2002921.519, 11651201.1..."
1,Existing,230,transmission_line,1.0,World Bank,Laos,None,None,None,None,None,line,"LINESTRING (11642041.532 2064458.971, 11637642...","POLYGON ((11637741.737 2064075.819, 11637193.0..."
2,Existing,230,transmission_line,2.0,World Bank,Laos,None,None,None,None,None,line,"LINESTRING (11895120.2 1689115.931, 11894570.3...","POLYGON ((11894481.255 1691616.252, 11852218.4..."
3,Existing,115,transmission_line,3.0,World Bank,Laos,None,None,None,None,None,line,"LINESTRING (11417899.886 2043628.146, 11431921...","POLYGON ((11431917.506 2044306.41, 11431935.18..."
4,Existing,115,transmission_line,4.0,World Bank,Laos,None,None,None,None,None,line,"LINESTRING (11417899.886 2043628.146, 11432471...","POLYGON ((11432490.024 2040979.482, 11432509.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,Existing,22,None,NaN,World Bank,None,None,None,None,None,None,line,"LINESTRING (11532976.034 2090761.016, 11536981...","POLYGON ((11537052.726 2086775.9, 11537065.71 ..."
423,Existing,22,None,NaN,World Bank,None,None,None,None,None,None,line,"LINESTRING (11538373.443 2088607.412, 11537840...","POLYGON ((11537743.256 2090921.908, 11537740.7..."
424,Existing,22,None,NaN,World Bank,None,None,None,None,None,None,line,"LINESTRING (11396008.1 2094168.895, 11397829.5...","POLYGON ((11397733.747 2100286.154, 11397740.7..."
425,Existing,22,None,NaN,World Bank,None,None,None,None,None,None,line,"LINESTRING (11369167.465 2255664.932, 11361572...","POLYGON ((11361655.11 2244413.017, 11361642.56..."


In [645]:
def assess_damage_pg(country_code,pg_infra,hazard_type):

    # load curves and maxdam
    curves,maxdam = load_curves_maxdam(vul_curve_path,hazard_type)
    #curves['line'] = 1 # remove this when things work!
    
    # read infrastructure data:
    pg_lines,pg_points = pg_infra
    
    if hazard_type=='tc':
        # read wind data
        climate_models = ['','_CMCC-CM2-VHR4','_CNRM-CM6-1-HR','_EC-Earth3P-HR','_HadGEM3-GC31-HM']
        df_ds = open_storm_data(country_code)

        # remove assets that will not have any damage
        pg_lines = pg_lines.loc[pg_lines.asset != 'cable'].reset_index(drop=True)
    
    elif hazard_type=='fl':
        # read flood data
        climate_models = ['historical','rcp8p5']
        df_ds = open_flood_data(country_code) 
    
    #calculate damaged lines/polygons/points in loop by climate_model
    damaged_lines = {}
    damaged_points = {}
    
    # calculate damaged lines/polygons/points in loop by climate_model
    for climate_model in climate_models:
        
        if hazard_type == 'tc':
            return_periods = ['1_1{}'.format(climate_model),'1_2{}'.format(climate_model),'1_5{}'.format(climate_model),'1_10{}'.format(climate_model),
                              '1_25{}'.format(climate_model),'1_50{}'.format(climate_model),'1_100{}'.format(climate_model),
                              '1_250{}'.format(climate_model),'1_500{}'.format(climate_model),'1_1000{}'.format(climate_model)]
        elif hazard_type == 'fl':
            return_periods = ['rp0001','rp0002','rp0005','rp0010','rp0025','rp0050','rp0100','rp0250','rp0500','rp1000']  

        # assess damage for lines
        overlay_lines = pd.DataFrame(overlay_hazard_assets(df_ds[climate_model],pg_lines).T,
                                     columns=['asset','hazard_point'])
        
        if len(overlay_lines) == 0:
            damaged_lines[climate_model] = pd.DataFrame()
        
        else:
            collect_line_damages = []
            for asset in tqdm(overlay_lines.groupby('asset'),total=len(overlay_lines.asset.unique()),
                              desc='polyline damage calculation for {} {} ({})'.format(country_code,hazard_type,climate_model)):
                for return_period in return_periods:
                    collect_line_damages.append(get_damage_per_asset_per_rp(asset,
                                                                           df_ds[climate_model],
                                                                           pg_lines,
                                                                           curves,
                                                                           maxdam,
                                                                           return_period,
                                                                           country_code))

            get_asset_type_line = dict(zip(pg_lines.index,pg_lines.asset))

            if hazard_type == 'fl':
                results = pd.DataFrame(collect_line_damages,columns=['rp','asset','curve','meandam','lowerdam','upperdam'])
            elif hazard_type == 'tc':
                results = pd.DataFrame([item for sublist in collect_line_damages for item in sublist],
                                       columns=['rp','asset','curve','meandam','lowerdam','upperdam'])

            results['asset_type'] = results.asset.apply(lambda x : get_asset_type_line[x])

            damaged_lines[climate_model] = results.groupby(['rp','curve','asset_type']).sum().drop(['asset'], axis=1).reset_index()

        # assess damage for points
        overlay_points = pd.DataFrame(overlay_hazard_assets(df_ds[climate_model],pg_points).T,
                                      columns=['asset','hazard_point'])
        
        if len(overlay_points) == 0:
            damaged_points[climate_model] = pd.DataFrame()
                          
        else:
            collect_point_damages = []
            for asset in tqdm(overlay_points.groupby('asset'),total=len(overlay_points.asset.unique()),
                              desc='point damage calculation for {} {} ({})'.format(country_code,hazard_type,climate_model)):
                for return_period in return_periods:
                    collect_point_damages.append(get_damage_per_asset_per_rp(asset,
                                                                            df_ds[climate_model],
                                                                            pg_points,
                                                                            curves,
                                                                            maxdam,
                                                                            return_period,
                                                                            country_code))

            get_asset_type_point = dict(zip(pg_points.index,pg_points.asset))

            if hazard_type == 'fl':
                results = pd.DataFrame(collect_point_damages ,columns=['rp','asset','curve','meandam','lowerdam','upperdam'])
            elif hazard_type == 'tc':
                results = pd.DataFrame([item for sublist in collect_point_damages for item in sublist],
                                       columns=['rp','asset','curve','meandam','lowerdam','upperdam'])

            results['asset_type'] = results.asset.apply(lambda x : get_asset_type_point[x])    

            damaged_points[climate_model] = results.groupby(['rp','curve','asset_type']).sum().drop(['asset'], axis=1).reset_index()

    return damaged_lines,damaged_points

In [ ]:
%%time
pg_infra = open_pg_data('LAO')
pg_damage_infra = assess_damage_pg('LAO',pg_infra,'tc')
pg_damage_infra

In [86]:
pg_damage_infra

({'_CMCC-CM2-VHR4':                       rp curve asset_type  asset       meandam      lowerdam  \
  0   1_1000_CMCC-CM2-VHR4  W5_1       line   6786  3.282313e+08  2.461735e+08   
  1   1_1000_CMCC-CM2-VHR4  W5_2       line   6786  3.474282e+08  2.605711e+08   
  2   1_1000_CMCC-CM2-VHR4  W5_3       line   6786  3.474282e+08  2.605711e+08   
  3    1_100_CMCC-CM2-VHR4  W5_1       line   6786  3.282313e+08  2.461735e+08   
  4    1_100_CMCC-CM2-VHR4  W5_2       line   6786  3.474282e+08  2.605711e+08   
  5    1_100_CMCC-CM2-VHR4  W5_3       line   6786  3.474282e+08  2.605711e+08   
  6     1_10_CMCC-CM2-VHR4  W5_1       line   6786  2.813719e+08  2.110289e+08   
  7     1_10_CMCC-CM2-VHR4  W5_2       line   6786  3.181328e+08  2.385996e+08   
  8     1_10_CMCC-CM2-VHR4  W5_3       line   6786  3.015049e+08  2.261287e+08   
  9      1_1_CMCC-CM2-VHR4  W5_1       line   6786  2.813719e+08  2.110289e+08   
  10     1_1_CMCC-CM2-VHR4  W5_2       line   6786  3.181328e+08  2.385996e+08  

In [641]:
def country_analysis_pg(country_code,hazard_type): #
    """_summary_

    Args:
        country_code (_type_): _description_
        hazard_type (str, optional): _description_. Defaults to 'OSM'.

    Returns:
        _type_: _description_
    """
    
    # extract infrastructure data from OSM
    pg_infra = open_pg_data(country_code)

    # assess damage to wind storms
    pg_damage_infra = assess_damage_pg(country_code,pg_infra,hazard_type)
    
    if hazard_type=='tc':
        climate_models = ['','_CMCC-CM2-VHR4','_CNRM-CM6-1-HR','_EC-Earth3P-HR','_HadGEM3-GC31-HM']
    elif hazard_type=='fl':
        climate_models = ['historical','rcp8p5']
    
    line_risk = {}
    plant_risk = {}
    substation_risk = {}
    tower_risk = {}
    pole_risk = {}
    
    for i in range(len(pg_damage_infra)):
        for climate_model in climate_models:
            df = pg_damage_infra[i][climate_model]
            
            if len(df) == 0:
                print("No {}_{} risk of infra_type {} in {}".format(hazard_type,climate_model,i,country_code))

            else:
                with pd.ExcelWriter(os.path.join(output_path,'damage','{}_{}_pg_{}_damage_{}'.format(country_code,climate_model,hazard_type,i)+'.xlsx')) as writer:
                    df.to_excel(writer)

                if hazard_type == 'tc':
                    df['rp'] = df['rp'].replace(['1_1{}'.format(climate_model),'1_2{}'.format(climate_model),'1_5{}'.format(climate_model),
                                                 '1_10{}'.format(climate_model),'1_25{}'.format(climate_model),'1_50{}'.format(climate_model),
                                                 '1_100{}'.format(climate_model),'1_250{}'.format(climate_model),'1_500{}'.format(climate_model),
                                                 '1_1000{}'.format(climate_model)],
                                                [1,0.5,0.2,0.1,0.04,0.02,0.01,0.004,0.002,0.001])

                elif hazard_type == 'fl':
                    df['rp'] = df['rp'].replace(['rp0001','rp0002','rp0005','rp0010','rp0025','rp0050','rp0100','rp0250','rp0500','rp1000'],
                                                [1,0.5,0.2,0.1,0.04,0.02,0.01,0.004,0.002,0.001])

                #assess risk for power lines
                if i == 0:
                    loss_list = df.meandam.values.tolist()
                    RPS = df.rp.values.tolist()
                    line_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                #assess risk for power plants, substations, power towers and power poles
                elif i == 1:
                    loss_list = df.loc[df['asset_type'] == 'plant']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'plant']
                    RPS = RPS.rp.values.tolist()
                    plant_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                    loss_list = df.loc[df['asset_type'] == 'substation']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'substation']
                    RPS = RPS.rp.values.tolist()
                    substation_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                    loss_list = df.loc[df['asset_type'] == 'power_tower']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'power_tower']
                    RPS = RPS.rp.values.tolist()
                    tower_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])

                    loss_list = df.loc[df['asset_type'] == 'power_pole']
                    loss_list = loss_list.meandam.values.tolist()
                    RPS = df.loc[df['asset_type'] == 'power_pole']
                    RPS = RPS.rp.values.tolist()
                    pole_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])
    
    return line_risk,plant_risk,substation_risk,tower_risk,pole_risk

In [642]:
pg_damage_infra = country_analysis_pg('LAO','fl')

Loading historical coastal flood data ...
Loading future coastal flood data ...


polyline damage calculation for LAO fl (historical): 0it [00:00, ?it/s]
point damage calculation for LAO fl (historical): 0it [00:00, ?it/s]
polyline damage calculation for LAO fl (rcp8p5): 0it [00:00, ?it/s]
point damage calculation for LAO fl (rcp8p5): 0it [00:00, ?it/s]


KeyError: 'historical'

In [ ]:
pg_damage_infra

# Save results of risk

In [32]:
def risk_output(country_code,hazard_type,infra_type):
    
    if infra_type == 'osm':
        total_risk = pd.DataFrame(country_analysis_osm(country_code,hazard_type))
        total_risk.to_excel(os.path.join(output_path,'risk','{}_{}_{}_risk'.format(country_code,infra_type,hazard_type)+'.xlsx'))
    
    elif infra_type == 'gov':
        total_risk = pd.DataFrame(country_analysis_pg(country_code,hazard_type))
        total_risk.to_excel(os.path.join(output_path,'risk','{}_{}_{}_risk'.format(country_code,infra_type,hazard_type)+'.xlsx'))
    
    return total_risk

In [33]:
total_risk = risk_output('LAO','fl','osm')

query is finished, lets start the loop


extract: 100%|██████████████████████████████████████████████████████████████████████| 439/439 [00:02<00:00, 155.19it/s]


query is finished, lets start the loop


extract: 100%|█████████████████████████████████████████████████████████████████████████| 33/33 [00:08<00:00,  3.83it/s]


query is finished, lets start the loop


extract: 100%|█████████████████████████████████████████████████████████████████| 45489/45489 [00:04<00:00, 9836.46it/s]


Loading historical coastal flood data ...
Loading future coastal flood data ...
No power lines exposed to fl in LAO
No power polygons exposed to fl in LAO
No power points exposed to fl in LAO
No power lines exposed to fl in LAO
No power polygons exposed to fl in LAO
No power points exposed to fl in LAO
No risk: LAO historical
No risk: LAO rcp8p5
No risk: LAO historical
No risk: LAO rcp8p5
No risk: LAO historical
No risk: LAO rcp8p5


In [36]:
total_risk = risk_output('TWN','fl','osm')
total_risk

query is finished, lets start the loop


extract: 100%|████████████████████████████████████████████████████████████████████| 2470/2470 [00:07<00:00, 335.43it/s]


query is finished, lets start the loop


extract: 100%|███████████████████████████████████████████████████████████████████████| 368/368 [00:18<00:00, 19.38it/s]


query is finished, lets start the loop


extract: 100%|█████████████████████████████████████████████████████████████| 1608621/1608621 [02:42<00:00, 9895.23it/s]


Loading historical coastal flood data ...
Loading future coastal flood data ...


polyline damage calculation for TWN fl (historical): 100%|█████████████████████████████| 58/58 [00:01<00:00, 31.32it/s]
polygon damage calculation for TWN fl (historical): 100%|██████████████████████████████| 60/60 [00:02<00:00, 20.43it/s]
point damage calculation for TWN fl (rcp8p5): 100%|██████████████████████████████████| 392/392 [00:06<00:00, 59.54it/s]


,historical,rcp8p5
0,1.054249e+05,1.369007e+05
1,1.956667e+09,2.598843e+09
2,5.313280e+06,6.924929e+06
3,1.452217e+05,1.970477e+05
4,0.000000e+00,1.953943e-01


In [356]:
osm_damage_infra[1]['historical'].loc[osm_damage_infra[1]['historical']['asset_type'] == 'plant']

,rp,curve,asset_type,meandam,lowerdam,upperdam
0,1.000,plant,plant,1.658665e+09,4.523633e+08,2.261816e+09
2,0.500,plant,plant,1.730781e+09,4.720311e+08,2.360156e+09
4,0.200,plant,plant,2.388258e+09,6.513430e+08,3.256715e+09
6,0.100,plant,plant,2.552940e+09,6.962564e+08,3.481282e+09
8,0.040,plant,plant,2.767454e+09,7.547601e+08,3.773801e+09
10,0.020,plant,plant,2.921201e+09,7.966912e+08,3.983456e+09
12,0.010,plant,plant,3.080762e+09,8.402079e+08,4.201039e+09
14,0.004,plant,plant,3.356950e+09,9.155320e+08,4.577660e+09
16,0.002,plant,plant,3.528684e+09,9.623684e+08,4.811842e+09
18,0.001,plant,plant,3.698459e+09,1.008671e+09,5.043354e+09


In [336]:
osm_damage_infra[0]['historical'].loc[:,"rp"]

0     1.000
1     1.000
2     1.000
3     0.500
4     0.500
5     0.500
6     0.200
7     0.200
8     0.200
9     0.100
10    0.100
11    0.100
12    0.040
13    0.040
14    0.040
15    0.020
16    0.020
17    0.020
18    0.010
19    0.010
20    0.010
21    0.004
22    0.004
23    0.004
24    0.002
25    0.002
26    0.002
27    0.001
28    0.001
29    0.001
Name: rp, dtype: float64

In [ ]:
"""
def clip_gridfinder(country_code):
    base_map_path = os.path.join(data_path,'base_map')

    cty_boundary_path = os.path.join(base_map_path,'gadm41_{}.gpkg'.format(country_code))
    cty_boundary = gpd.read_file(cty_boundary_path)
    #mask = pd.DataFrame(mask.copy())
    #mask.geometry = pygeos.from_shapely(mask.geometry)
    #mask['geometry'] = reproject(mask)

    gridfinder_path = r'C:\Users\mye500\OneDrive - Vrije Universiteit Amsterdam\01_Research-Projects\01_risk_assessment\PG_data\gridfinder\grid.gpkg'
    gridfinder = gpd.read_file(gridfinder_path)
    #gridfinder = pd.DataFrame(gridfinder.copy())
    #gridfinder.geometry = pygeos.from_shapely(gridfinder.geometry)
    #gridfinder['geometry'] = reproject(gridfinder)

    clipped = gpd.clip(gridfinder,cty_boundary)

    return clipped

clip_gridfinder('TWN')
"""

In [570]:
#def monetary_risk(RPS,loss_list):
def monetary_risk(osm_damage_infra,hazard_type):
    """
    Calculates the monetary risk based on the return periods and the losses.
    Arguments:
        *RPS* : List of return periods (in years) for which the losses are calculated.
        *loss_list* : List of losses (in euro) for each return period.
    Returns:
        *total_risk* : Returns the total risk for the area
    """
    
    # assess damage to hazard_type
    #osm_damage_infra = country_analysis_osm(country_code,hazard_type)
    
    if hazard_type=='tc':
        climate_models = ['','_CMCC-CM2-VHR4','_CNRM-CM6-1-HR','_EC-Earth3P-HR','_HadGEM3-GC31-HM']
        total_risk = pd.DataFrame(columns=[climate_models])

    elif hazard_type=='fl':
        climate_models = ['historical','rcp8p5']
        total_risk = pd.DataFrame(columns=[climate_models])

    line_risk = {}
    plant_risk = {}
    substation_risk = {}
    tower_risk = {}
    pole_risk = {}
    total_risk = {}
    osm_risk = {}
    
    asset_types = ['line','minor_line','cable','plant','substation','power_tower','power_pole']
    
    for climate_model in climate_models:
        for i in range(len(osm_damage_infra)):
            for asset_type in asset_types:
                
                df = osm_damage_infra[i][climate_model]
                df['rp'] = df['rp'].replace(['rp0001','rp0002','rp0005','rp0010','rp0025','rp0050','rp0100','rp0250','rp0500','rp1000'],
                                            [1,0.5,0.2,0.1,0.04,0.02,0.01,0.004,0.002,0.001])
                
                loss_list = df.loc[df['asset_type'] == asset_type]
                RPS = loss_list.rp.values.tolist()
                #RPS = df.loc[df['asset_type'] == 'asset_type']
                
                loss_list = loss_list.meandam.values.tolist()
                print(loss_list)
                print(RPS)
                
                osm_risk[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])
                
                #'{}_risk'.format(asset_type)[climate_model] = integrate.simps(y=loss_list[::-1], x=RPS[::-1])
                
                #total_risk[climate_model] = total_risk.update(line_risk)
                #print(integrate.simps(y=loss_list_line[::-1], x=RPS[::-1]))
                #print(line_risk[climate_model])

                
                #total_risk[climate_model] = total_risk.update(plant_risk)
                #total_risk.append(plant_risk[climate_model])

    #total_risk = pd.DataFrame(total_risk[climate_model])
    """
    if hazard_type == 'tc':
        total_risk = pd.DataFrame({'historical':pd.Series(line_risk[0]['historical']),'rcp8p5':pd.Series(line_risk[1]['rcp8p5'])})
    
    elif hazard_type == 'fl':
        total_risk = pd.DataFrame({'historical':pd.Series(line_risk[0]['historical']),'rcp8p5':pd.Series(line_risk[1]['rcp8p5'])})
    """        
    return osm_risk#line_risk,plant_risk,substation_risk,tower_risk,pole_risk,total_risk#line_risk,plant_risk,substation_risk,tower_risk,pole_risk #integrate.simps(y=loss_list[::-1], x=RPS[::-1])